## Import Required Libraries

- `heapq`: Provides priority queue for A Star Algorithm
- `queue`: Provides  queue for Breadth-First Search
- `graphviz`: Used for graph visualization, helpful for visualizing data structures.
- `uuid`: Generates universally unique identifiers (UUIDs).



In [1]:
import heapq
import copy
import graphviz as g
import uuid
import random
import queue
import math
from IPython.display import display

##**Problem**
The Problem class represents the task environment for an agent, defining the initial state, goal state(s), and methods to support environment formulation. It includes the following methods:

- **GetInitialState (`self`, `initial_state`)**: The starting state of the problem.
- **Goal states (`self`, `goal_states`)**: The set of states that satisfy the goal condition.
- **isGoalState (`self`, `state`)**: Determines whether a given state is a goal state.
- **getSuccessors (`self`, `state`)**: Returns possible next states from the current state.
- **getActionCost (`self`, `state`, `action`)**: Returns the cost of taking an action from a state.(For 8-puzzle problem the cost of action is 1)


In [2]:
class Problem:
    def __init__(self, initial_state, goal_states):
        self.initial_state = initial_state
        self.goal_states = goal_states

    def getInitialState(self):
        return self.initial_state

    def isGoalState(self, state):
        return state in self.goal_states

    def getSuccessors(self, state):
        node = Node(state)
        return node.getSuccessors()

    def getActionCost(self, state, action):
        node = Node.Node(state)
        return node.getActionCost()

    def get_goal_states(self):
        return self.goal_states

    def get_initial_state(self):
        return self.initial_state

##**State**

The `State` class represents a board configuration in a tile-based puzzle. It includes the following methods:
- **GetActions (`self`)**: List of possible actions of the current state.
- **GetActionCost (`self`, `state`, `action`)**: Get action cost
- **getSuccessor (`self`, `action`)**: Returns the resulting successor after applying the action
- **are_adjacent (`getSuccessors`)**, **auto_swap_if_adjacency(`self`)**: Implements logic for swap the tiles if they are in pair (1 and 3 or 2 and 4)



In [3]:
class State():
    def __init__(self, board):
        self.board = board

    def __eq__(self, other):
        return self.board == other.board

    def __hash__(self):
        return hash(str(self.board))

    def __str__(self):
        return str(self.board)

    def get_board(self):
        return self.board

    def set_board(self, board):
        self.board = board

    def get_blank_position(self):
        for i in range(len(self.get_board())):
            for j in range(len(self.get_board()[0])):
                if self.board[i][j] == 0:
                    return (i, j)

    def swap(self, pos1, pos2):
        i1, j1 = pos1
        i2, j2 = pos2
        self.board[i1][j1], self.board[i2][j2] = self.board[i2][j2], self.board[i1][j1]

    '''Get the possible actions of the current state'''
    def getActions(self):
        actions = []
        blank_position = self.get_blank_position()
        if blank_position[0] > 0:
            actions.append('U')
        if blank_position[0] < len(self.get_board()) - 1:
            actions.append('D')
        if blank_position[1] > 0:
            actions.append('L')
        if blank_position[1] < len(self.get_board()) - 1:
            actions.append('R')
        return actions

    '''Cost of action is 1'''
    def getActionCost(self, state, action):
        return 1

    '''Return successor with action'''
    def getSuccessor(self, action):
        blank_position = self.get_blank_position()
        new_state = copy.deepcopy(self)

        if 'U' == action:
            new_state.swap(blank_position, (blank_position[0] - 1, blank_position[1]))
        elif 'D' == action:
            new_state.swap(blank_position, (blank_position[0] + 1, blank_position[1]))
        elif 'L' == action:
            new_state.swap(blank_position, (blank_position[0], blank_position[1] - 1))
        elif 'R' == action:
            new_state.swap(blank_position, (blank_position[0], blank_position[1] + 1))


        new_state.auto_swap_if_adjacency()
        return new_state


    def are_adjacent(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1]) == 1



    def auto_swap_if_adjacency(self):
        '''using dictionary to store position of each number in the board'''
        position = {}
        for i in range(len(self.get_board())):
            for j in range(len(self.get_board()[0])):
                position[self.board[i][j]] = (i, j)

        ''' swap if cell 1 and cell 3 are adjacent'''
        if self.are_adjacent(position[1], position[3]):
            self.swap(position[1], position[3])

        '''swap if cell 2 and cell 4 are adjacent'''
        if self.are_adjacent(position[2], position[4]):
            self.swap(position[2], position[4])


## **Node**

The `Node` class represents a state in the search tree. Each node stores information about the state, the action taken to reach it, its parent, and heuristic values used for informed search algorithms like A*.It includes the following methods:

- **__str__(`self`)**: Returns a string representation of the board, including `G` and `H` values.
- **__lt__(`self`, `other`)**: Compares two nodes based on their `f(x) = G + H` value for A* search.
- **get_id(`self`)**: Generates a unique ID for visualization based on the state's hash.
- **getSuccessors(`self`)**: Generates and returns a list of successor nodes by applying all possible actions.
- **draw(`self`, `dot`, `color='black'`, `drawn_edges=None`)**: Visualizes the node in a graph structure using Graphviz, drawing edges to its `parent`.



In [4]:
class Node:
    def __init__(self, state, action = None, parent = None, G =None, H=None):
        self.state = state
        self.action = action
        self.parent = parent
        self.G = G
        self.H = H
        self.ID = str(self)

    ''''''
    def __str__(self):
        board_str = '\n'.join(''.join(str(num) if num != 0 else '_' for num in row) for row in self.get_state().get_board())
        return f"{board_str}\n g_value:{self.get_G()}\n h_value: {self.H}"

    def __lt__(self, other):
        """ Compare the f_x for AStar """
        return (self.G + self.H) < (other.G + other.H)
    def get_state(self):
        return self.state

    def get_action(self):
        return self.action

    def get_G(self):
        return self.G

    def set_G(self, G):
        self.G = G

    def get_parent(self):
        return self.parent

    def get_id(self):
        """ Dùng ID cố định dựa trên state để vẽ đồ thị """
        return str(hash(str(self.state)))

    #For visualize the result
    def get_str_node(self):
        return str(self)

    def getActionCost(self):
        return 1

    def set_H(self, H):
        self.H = H

    def get_H(self):
        return self.H

    def set_parent(self, parent):
        self.parent = parent

    #Get all successors of a node
    def getSuccessors(self):
        successors = []
        state_of_node = self.get_state()
        for action in state_of_node.getActions():
            new_state = state_of_node.getSuccessor(action)
            cost = new_state.getActionCost(state_of_node, action)
            successors.append(Node(new_state, action, self, cost))
        return successors


    def draw(self, dot, color='black', drawn_edges=None):
        dot.node(self.get_id(), self.get_str_node(), shape="square", color=color)
        if self.get_parent() is not None:
            dot.edge(self.get_parent().get_id(), self.get_id(), label=self.get_action(), color=color)


## Heuristic

- `Num_linear_conflict(current_board, goal_board)`: Counts linear conflicts (tiles blocking each other) in rows and columns.
- `EnhancedManhattan(state, goalStates)`: Computes Manhattan distance plus a conflict penalty for more accurate heuristics.
- `getHeuristic(state, goalStates)`: Calls the selected heuristic function (`EnhancedManhattan` or `EnhancedMissplace`).


In [5]:
# manhattan 
# misplaced
# linear conflict
# linear conflict + manhattan
# linear conflict + misplaced
# pattern database
# ignore_swap_hihi
# bfs

class Heuristic:
    def __init__(self, type):
        self.heuristic_type = type
        self.Heuristics =    {
                                "EnhancedManhattan": self.EnhancedManhattan,
                                "EnhancedMissPlace": self.EnhancedMissplace,
                                "PatternDataBase": self.PatternDataBase,
                             }
        self.heuristic_func = self.Heuristics[self.heuristic_type]
        '''For PaternDatabse Heuristic'''
        self.pdbList = []

    '''Conflict: Number of tile pairs in which one tile prevent other from reaching the goal position'''
    @staticmethod
    def Num_linear_conflict(current_board, goal_board):
        number_of_conflicts = 0

        '''Look-up table for permutation can cause the conflict'''
        look_up_conflict = lambda a,b,c: [[b,a,c], [c,b,a], [a,c,b]]

        ''' Caculate number of confilcts in rows'''
        for row in range(len(current_board)):
            current_row = current_board[row]
            goal_row = goal_board[row]
            a,b,c = goal_row[0], goal_row[1], goal_row[2]
            potential_conflict_rows = look_up_conflict(a,b,c)
            if current_row in potential_conflict_rows:
                if current_row == potential_conflict_rows[1]:
                    number_of_conflicts += 2
                    continue
                number_of_conflicts += 1

        ''' Caculate the number of conflict in columns'''
        for col in range(len(current_board)):
            current_row = [current_board[row][col] for row in range(len(current_board))]
            goal_row= [goal_board[row][col] for row in range(len(goal_board))]
            a,b,c = goal_row[0], goal_row[1], goal_row[2]
            potential_conflict_rows = look_up_conflict(a,b,c)
            if current_row in potential_conflict_rows:
                if current_row == potential_conflict_rows[1]:
                    number_of_conflicts += 2
                    continue
                number_of_conflicts += 1
        return number_of_conflicts

    '''A heuristic that combines the traditional Manhattan distance with the number of conflicts
    (Clarifies cases where two states have the same Manhattan distance but different tile positions in a row or column).'''
    def EnhancedManhattan(self, state, goalStates):
      list_distance = []
      for goal in goalStates:

          board_goal = goal.get_board()
          goal_pos = {board_goal[i][j]: (i, j) for i in range(len(board_goal)) for j in range(len(board_goal))}


          distance = 0
          board_state = state.get_board()
          for i in range(len(board_state)):
              for j in range(len(board_state)):
                  tile = board_state[i][j]
                  if tile != 0:
                      gx, gy = goal_pos[tile]
                      distance += abs(gx - i) + abs(gy - j)


          conflict = Heuristic.Num_linear_conflict(board_state, board_goal)
          list_distance.append(distance + 2 * conflict)
      return min(list_distance)

    @staticmethod
    def EnhancedManhattanWithAGoal( state, goalState):
        # Lấy bảng đích
        board_goal = goalState.get_board()
        goal_pos = {board_goal[i][j]: (i, j) for i in range(len(board_goal)) for j in range(len(board_goal))}

        # Tính Manhattan
        distance = 0
        board_state = state.get_board()
        for i in range(len(board_state)):
            for j in range(len(board_state)):
                tile = board_state[i][j]
                if tile != 0:
                    gx, gy = goal_pos[tile]
                    distance += abs(gx - i) + abs(gy - j)

        # Tính số conflict với goal hiện tại
        conflict = Heuristic.Num_linear_conflict(board_state, board_goal)

        # Trả về tổng Manhattan và conflict
        return distance + 2 * conflict

    def EnhancedMissplace(self, state, goalStates):
        min_misplaced = 9
        for goalState in goalStates:
            misplaced = 0
            board_state = state.get_board()
            board_goal = goalState.get_board()


            for i in range(len(board_state)):
                for j in range(len(board_state)):
                    if board_state[i][j] != board_goal[i][j]:
                        misplaced += 1


            conflict = Heuristic.Num_linear_conflict(board_state, board_goal)
            min_misplaced = min(min_misplaced, misplaced + 2 * conflict)

        return min_misplaced





# Procedure BUILD_FOUR_PDBS(goals[4]):
#     pdb_list ← ∅
#     For each goal g in goals:
#         pdb_g ← BUILD_PDB_FROM_GOAL(g)      // BFS để xây PDB cho goal g
#         pdb_list.add(pdb_g)
#     return pdb_list

# Procedure BUILD_PDB_FROM_GOAL(goal):
#     Initialize an empty map pdb           // pdb[state] = cost ngắn nhất từ state đến goal
#     Create a queue Q
#     pdb[goal] ← 0
#     Enqueue(goal, Q)
#     while Q is not empty:
#         current ← Dequeue(Q)
#         current_cost ← pdb[current]
#         successors ← GET_SUCCESSORS(current)  // sinh tất cả successor (có auto swap)
#         for each (succ, stepCost) in successors:
#             if succ not in pdb:
#                 pdb[succ] ← current_cost + stepCost
#                 Enqueue(succ, Q)
#     return pdb

    def PatternDataBase(self, state, goalStates):
        """
        Hàm được gọi mỗi lần cần tính heuristic PDB cho 'state'.
        Ta đã có self.pdbList = [pdb_goal1, pdb_goal2, pdb_goal3, pdb_goal4]
        Mỗi pdb_goalX là dict: { state_tupla -> cost }.

        => Tính cost = min( pdb_goalX.get(state_tupla, infinity) )
        """
        if not self.pdbList:
            # Nếu chưa build, trả về 0 hoặc Infinity tuỳ
            return 0
        state_key = tuple(num for row in state.get_board() for num in row)
        best_cost = math.inf
        for pdb_dict in self.pdbList:
            cost = pdb_dict.get(state_key, math.inf)
            if cost < best_cost:
                best_cost = cost
        return best_cost

    def PatternDataBaseBuildAll(self, goalStates):
        """
        Xây 4 PDB, mỗi PDB_i ứng với goalStates[i].
        Trả về list 4 dict.
        """
        pdb_list = []
        for goal in goalStates:
            pdb_g = self.buildPDBFromGoal(goal)
            pdb_list.append(pdb_g)
        return pdb_list

    def buildPDBFromGoal(self, goal):
        """
        BFS từ goal, duyệt toàn bộ state reachable (theo quy tắc di chuyển + auto swap),
        ghi pdb[state_tuple] = cost từ state -> goal.
        """
        pdb = {}
        Q = queue.Queue()

        # Chuyển goal thành key (tuple)
        goal_key = tuple(num for row in goal.get_board() for num in row)

        pdb[goal_key] = 0
        Q.put(goal)

        while not Q.empty():
            current_state = Q.get()
            current_cost_key = tuple(num for row in current_state.get_board() for num in row)
            current_cost = pdb[current_cost_key]

            current_node = Node(current_state)
            for successor in current_node.getSuccessors():
                succ_key = tuple(num for row in successor.get_state().get_board() for num in row)
                if succ_key not in pdb:
                    pdb[succ_key] = current_cost + successor.getActionCost()
                    Q.put(successor.get_state())

        return pdb
    def getHeuristic(self, state, goalStates):
        return self.heuristic_func(state, goalStates)

In [6]:
goal_states = []
# goal_states.append(State([[1, 2, 3], [4, 5, 6], [7, 8, 0]]))

# goal_states.append(State([[8, 7, 6], [5, 4, 3], [2, 1, 0]]))

# goal_states.append(State([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

# goal_states.append(State([[0, 8, 7], [6, 5, 4], [3, 2, 1]]))

# Giả sử bạn đã có H.pdbList (4 dict PDB)
H = Heuristic(type="PatternDataBase")  # Or any code that fills H.pdbList
n = H.PatternDataBaseBuildAll(goal_states)

In [7]:
import pickle
import os

# Đường dẫn thư mục trên máy tính nơi bạn muốn lưu file
# Ví dụ: Lưu trong thư mục "MyPDB" nằm cùng với file Python của bạn
save_dir = './MyPDB/'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(save_dir, exist_ok=True)

# Giả sử H.pdbList đã được khởi tạo và chứa dữ liệu cần lưu
i = 1
for goal_pdb in n:
    
    filename = f'DatabaseForGoal{i}.pkl'
    fullpath = os.path.join(save_dir, filename)  # Nối đường dẫn đầy đủ

    with open(fullpath, 'wb') as f:
        pickle.dump(goal_pdb, f)

    print(f"Saved file: {fullpath}")
    i += 1


In [8]:
# import os
# import pickle

# # Đường dẫn chứa các file pickle
# save_dir = './MyPDB/'  # Đường dẫn tới thư mục chứa các file pickle đã lưu

# # Tạo dictionary để chứa dữ liệu load được
# loaded_data = []

# # Giả sử tên file là DatabaseForGoal1.pkl, DatabaseForGoal2.pkl, DatabaseForGoal3.pkl, DatabaseForGoal4.pkl
# for i in range(1, 5):
#     filename = f'DatabaseForGoal{i}.pkl'
#     fullpath = os.path.join(save_dir, filename)
    
#     # Mở file ở chế độ đọc nhị phân và load dữ liệu
#     with open(fullpath, 'rb') as f:
#         data = pickle.load(f)
#         # Lưu vào dictionary với key tùy chọn, ví dụ: 'goal_1', 'goal_2', ...
#         loaded_data.append(data)
        
#     print(f"Loaded file: {fullpath}")

# # In ra nội dung của dictionary chứa dữ liệu đã load
# print("All data loaded:")
# print(loaded_data)


In [9]:
class Search:
    def __init__(self,heuristic: Heuristic, numberNodesDraw = None):
        self.heuristic = heuristic
        self.graph = None
        self.numberNodesDraw = numberNodesDraw if numberNodesDraw is not None else 0
        self.uniqueId = str(uuid.uuid4())[:8] #Id for visual graph

    '''Graph Search AStar Algorithm '''
    def AStar(self, problem):
        if self.numberNodesDraw:
            self.graph = self.graph = g.Digraph(name=f"Graph_{self.uniqueId}")
        expandedNodes = 0

        initial_state = problem.get_initial_state()
        if problem.isGoalState(initial_state):
            return [], expandedNodes

        initial_H  = self.heuristic.heuristic_func(initial_state, problem.get_goal_states())
        initial_node = Node(initial_state, H= initial_H, G = 0)

        frontier = []
        heapq.heappush(frontier, (0, initial_node))

        costSoFar = {initial_node.get_state() : 0}
        #explored = set()  (ko caanf thiet )

        while frontier:
            g_value, node = heapq.heappop(frontier)
            expandedNodes += 1


            '''Only draw nodes up to the allowed limit; the algorithm still runs normally.'''
            if self.numberNodesDraw is not None and expandedNodes <= self.numberNodesDraw:
                node.draw(self.graph)

            if problem.isGoalState(node.get_state()):
                return self.getPath(node), expandedNodes

            for successor in node.getSuccessors():
                stateOfSuccessor = successor.get_state()
                g_value = node.get_G() + successor.getActionCost()
                h_value = self.heuristic.heuristic_func(stateOfSuccessor, problem.get_goal_states())


                '''No need to compute f(x) = g(x) + h(x) explicitly as __lt__ is defined in Node.'''
                if stateOfSuccessor not in costSoFar or g_value < costSoFar[stateOfSuccessor]:
                    costSoFar[stateOfSuccessor] = g_value
                    successor.set_G(g_value)
                    successor.set_H(h_value)
                    successor.set_parent(node)
                    heapq.heappush(frontier, (g_value, successor))

        return None, None


    def getPath(self, node):
        path = []
        while node:
            path.append(node)
            if isinstance(node, Node):
                node = node.get_parent()
        return path[::-1]

    '''Visualize the AStar Algorithm'''
    def Visual(self, numberNodesDraw, problem):
        self.numberNodesDraw = numberNodesDraw
        path_nodes,_ = self.AStar(problem)
        display(g.Source(self.graph.source))


In [10]:
class BFS():
  def BFS(self, problem: Problem):
      expandedNodes = 0
      initial_state = problem.get_initial_state()
      if problem.isGoalState(initial_state):
          return [], expandedNodes

      initial_node = Node(initial_state, G=0)
      frontier = []
      frontier.append(initial_node)
      explored = set()

      while frontier:
          node = frontier.pop(0)
          expandedNodes += 1

          if problem.isGoalState(node.get_state()):
              return self.getPath(node), expandedNodes

          for successor in node.getSuccessors():
              stateOfSuccessor = successor.get_state()
              if stateOfSuccessor not in explored:
                  explored.add(stateOfSuccessor)
                  successor.set_parent(node)
                  frontier.append(successor)
      return None

  def getPath(self, node):
    path = []
    while node:
        path.append(node)
        if isinstance(node, Node):
            node = node.get_parent()
    return path[::-1]

In [11]:
# run bfs and store the list of action and total cost in a file and expanded node
# store the initial state and goal state in the file

def run_bfs(problem, initial_state, testcase_number=1):
    bfs = BFS()
    path, expanded_nodes = bfs.BFS(problem)
    if path is None:
        print("No solution found.")
        return

    # Lấy danh sách các hành động (loại bỏ node đầu tiên không có hành động)
    actions = [node.get_action() for node in path if node.get_action() is not None]
    # Lấy trạng thái cuối cùng
    
    if problem.isGoalState(initial_state):
        final_state = initial_state.get_board()
    else:
        final_state = path[-1].get_state().get_board()
    
    # Mở file ở chế độ append để ghi thêm kết quả của testcase mới
    save_path = "bfs_result.txt"
    os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)

    with open(save_path, "a") as f:
        f.write(f"Testcase: {testcase_number}\n")
        f.write(f"Initial State: {initial_state.get_board()}\n")
        f.write(f"Goal State: {final_state}\n")
        f.write(f"Total Cost: {len(path) - 1}\n")
        f.write(f"Expanded Nodes: {expanded_nodes}\n")
        f.write("Actions: ")
        for action in actions:
            f.write(f"{action} -> ")
        f.write("Goal Reached\n")
        f.write("----------------------------------------------------\n\n")

    print(f"testcase number {testcase_number}")



In [12]:
def load_all_states(file_path):
    with open(file_path, 'rb') as f:
        states = pickle.load(f)
    return states

# Kiểm tra: Load lại các trạng thái đã lưu và in ra số lượng cũng như 5 trạng thái đầu tiên
loaded_states = load_all_states('./all_states_8_puzzles.pkl')


In [13]:
# 
goal_states = []
goal_states.append(State([[1, 2, 3], [4, 5, 6], [7, 8, 0]]))
goal_states.append(State([[8, 7, 6], [5, 4, 3], [2, 1, 0]]))
goal_states.append(State([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
goal_states.append(State([[0, 8, 7], [6, 5, 4], [3, 2, 1]]))

numberLoadState = len(loaded_states)

for i in range(275000,300000):
    initial_state = State(loaded_states[i])
    problem = Problem(initial_state, goal_states)
    run_bfs(problem, initial_state, i)


testcase number 275000
testcase number 275001
testcase number 275002
testcase number 275003
testcase number 275004
testcase number 275005
testcase number 275006
testcase number 275007
testcase number 275008
testcase number 275009
testcase number 275010
testcase number 275011
testcase number 275012
testcase number 275013
testcase number 275014
testcase number 275015
testcase number 275016
testcase number 275017
testcase number 275018
testcase number 275019
testcase number 275020
testcase number 275021
testcase number 275022
testcase number 275023
testcase number 275024
testcase number 275025
testcase number 275026
testcase number 275027
testcase number 275028
testcase number 275029
testcase number 275030
testcase number 275031
testcase number 275032
testcase number 275033
testcase number 275034
testcase number 275035
testcase number 275036
testcase number 275037
testcase number 275038
testcase number 275039
testcase number 275040
testcase number 275041
testcase number 275042
testcase nu

KeyboardInterrupt: 

##Demo

In [ ]:
class Demo:
    def __init__(self, heuristic_choice, mode_choice, numberNodesDraw=None ):
        # heuristic_choice: '1' cho EnhancedManhattan, '2' cho EnhancedMissPlace
        # mode_choice: '1' để chạy mô phỏng có vẽ, '2' để chỉ trả kết quả đánh giá
        if heuristic_choice == '1':
            self.heuristic_type = "EnhancedManhattan"
        elif heuristic_choice == '2':
            self.heuristic_type = "EnhancedMissPlace"
        elif heuristic_choice == '3':
            self.heuristic_type = "PatternDataBase"
        else:
            raise ValueError("Lựa chọn heuristic không hợp lệ. Vui lòng chọn 1 hoặc 2.")

        self.mode_choice = mode_choice

        # Tạo trạng thái ban đầu
        self.initial_state = self.generate_random_state()

        # Thiết lập các trạng thái mục tiêu
        self.goal_states = []
        self.goal_states.append(State([[1, 2, 3], [4, 5, 6], [7, 8, 0]]))
        self.goal_states.append(State([[8, 7, 6], [5, 4, 3], [2, 1, 0]]))
        self.goal_states.append(State([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))
        self.goal_states.append(State([[0, 8, 7], [6, 5, 4], [3, 2, 1]]))

        # Tạo bài toán
        self.problem = Problem(self.initial_state, self.goal_states)

        # Tạo đối tượng tìm kiếm với heuristic đã chọn và số node cần vẽ (nếu có)
        self.search_algo = Search(Heuristic(self.heuristic_type), numberNodesDraw)

    @staticmethod
    def generate_random_state():
        numbers = list(range(9))
        random.seed(42)
        random.shuffle(numbers)
        board = [numbers[i:i+3] for i in range(0, 9, 3)]
        return State(board)

    def run(self):
        if self.mode_choice == '1':
            self.run_simulation()
        elif self.mode_choice == '2':
            self.run_evaluation()
        else:
            print("Lựa chọn chế độ không hợp lệ.")

    def run_simulation(self):
        # Chạy tìm kiếm A* và mô phỏng (với đồ thị)
        path, expanded_nodes = self.search_algo.AStar(self.problem)
        print("=== A* Search Simulation ===")
        print("Initial State:", self.initial_state.get_board())
        for i, node in enumerate(path, start=1):
            print(f"Step {i}: {node.get_state().get_board()}")
        print("Total cost:", len(path))
        print("Số node mở rộng:", expanded_nodes)
        # Vẽ đồ thị với số node cần vẽ (giá trị đã thiết lập)
        self.search_algo.Visual(self.search_algo.numberNodesDraw,self.problem)

    def run_evaluation(self):
     # Đánh giá Pattern Database heuristic (nếu được chọn)
        if self.heuristic_type == "PatternDataBase":
            # 1. Build Pattern Database trước khi chạy A*
            self.search_algo.heuristic.pdbList = self.search_algo.heuristic.PatternDataBaseBuildAll(self.problem.get_goal_states())

            # 2. Chạy A* với PDB heuristic
            path_pdb, expanded_pdb = self.search_algo.AStar(self.problem)
            print("Pattern Database Heuristic Search:")
            print("Initial State:", self.initial_state.get_board())
            for i, node in enumerate(path_pdb, start=1):
                h_value = self.search_algo.heuristic.PatternDataBase(node.get_state(), self.problem.get_goal_states()) # Sử dụng hàm PDB heuristic đã build
                print(f"Step {i}: {node.get_state().get_board()}, H_value: {h_value}")
            print("Total cost (Pattern Database):", len(path_pdb))
            print("Số node mở rộng (Pattern Database):", expanded_pdb)
            print("\n-----------------------------\n")
        # Đánh giá A* search
        astar_search = Search(Heuristic(self.heuristic_type))
        path_astar, expanded_astar = astar_search.AStar(self.problem)
        print("A* Search:")
        print("Initial State:", self.initial_state.get_board())
        for i, node in enumerate(path_astar, start=1):
            h_value = Heuristic.EnhancedManhattanWithAGoal(node.get_state(), path_astar[-1].get_state())
            print(f"Step {i}: {node.get_state().get_board()}, H_value: {h_value}")
        print("Total cost (A*):", len(path_astar))
        print("Số node mở rộng (A*):", expanded_astar)
        print("\n-----------------------------\n")
        # Đánh giá BFS search
        bfs_search = BFS()
        path_bfs, expanded_bfs = bfs_search.BFS(self.problem)
        print("BFS Search:")
        print("Initial State:", self.initial_state.get_board())
        for i, node in enumerate(path_bfs, start=1):
            print(f"Step {i}: {node.get_state().get_board()}")
        print("Total cost (BFS):", len(path_bfs))
        print("Số node mở rộng (BFS):", expanded_bfs)

# -------------------------------
# Giao diện chính (Main)
# -------------------------------

def main():

    print("Chọn ví dụ cho Heuristic:")
    print("1: EnhancedManhattan")
    print("2: EnhancedMissPlace")
    print("3: PatternDataBase")
    heuristic_choice = input("Nhập lựa chọn (1, 2 hoặc 3): ").strip()

    print("\nChọn chế độ:")
    print("1: Chạy mô phỏng (có vẽ mô phỏng)")
    print("2: Chỉ trả đánh giá kết quả")
    mode_choice = input("Nhập lựa chọn (1 hoặc 2): ").strip()
    print("Chọn số testcase: ")
    n = input("Nhập số testcase (số nguyên): ").strip()

    numberNodesDraw = 0
    if mode_choice == '1':
        # Yêu cầu người dùng nhập số node muốn vẽ (nếu có)
        try:
            numberNodesDraw = int(input("Nhập số node muốn vẽ (số nguyên): ").strip())
        except ValueError:
            numberNodesDraw = 0

    try:
        for i in range(int(n)):
            print(f"\nTest case {i+1}:")
            demo = Demo(heuristic_choice, mode_choice, numberNodesDraw)
            demo.run()
    except ValueError as e:
        print(e)

if __name__ == "__main__":
    main()

Chọn ví dụ cho Heuristic:
1: EnhancedManhattan
2: EnhancedMissPlace
3: PatternDataBase

Chọn chế độ:
1: Chạy mô phỏng (có vẽ mô phỏng)
2: Chỉ trả đánh giá kết quả
Chọn số testcase: 

Test case 1:
Pattern Database Heuristic Search:
Initial State: [[3, 6, 7], [4, 8, 2], [5, 0, 1]]
Step 1: [[3, 6, 7], [4, 8, 2], [5, 0, 1]], H_value: 13
Step 2: [[3, 6, 7], [4, 0, 2], [5, 8, 1]], H_value: 12
Step 3: [[3, 6, 7], [2, 4, 0], [5, 8, 1]], H_value: 13
Step 4: [[3, 6, 7], [2, 0, 4], [5, 8, 1]], H_value: 10
Step 5: [[3, 6, 7], [2, 8, 4], [5, 0, 1]], H_value: 9
Step 6: [[3, 6, 7], [2, 8, 4], [0, 5, 1]], H_value: 8
Step 7: [[3, 6, 7], [0, 8, 4], [2, 5, 1]], H_value: 7
Step 8: [[0, 6, 7], [3, 8, 4], [2, 5, 1]], H_value: 6
Step 9: [[6, 0, 7], [3, 8, 4], [2, 5, 1]], H_value: 5
Step 10: [[6, 8, 7], [3, 0, 4], [2, 5, 1]], H_value: 4
Step 11: [[6, 8, 7], [3, 5, 4], [2, 0, 1]], H_value: 3
Step 12: [[6, 8, 7], [3, 5, 4], [0, 2, 1]], H_value: 2
Step 13: [[6, 8, 7], [0, 5, 4], [3, 2, 1]], H_value: 1
Step 14: [

KeyboardInterrupt: 